In [1]:
from bs4 import BeautifulSoup
import requests
import os.path

In [2]:
def normalize(s):
    replacements = (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"),
                    ("ú", "u"), ("ñ","n"), ("  "," "),(" ","-"))
    s = s.strip()
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
        s = s.lower()
    return s

In [9]:
docentes = {}
url_artes = 'http://www.facartes.unal.edu.co/fa/docentes/index.php'
html_artes = requests.get(url_artes).text
arbol = BeautifulSoup(html_artes)
#print(arbol.find("option",selected=True).text)
escuelas = arbol.find_all("option")
#print(escuelas[0]['value'])

for escuela in escuelas:
    docente_dep = []
    new_url = url_artes+"?escuela="+escuela['value']
    html_artes = requests.get(new_url).text
    arbol = BeautifulSoup(html_artes)
    for i in arbol.find_all("span", {"class":"name"}):
        docente_dep.append(normalize(i.text))
    docentes[normalize(arbol.find("option",selected=True).text)] = docente_dep
docentes["artes-plasticas-y-visuales"] = docentes.pop("artes-plasticas")

for key, val in docentes.items():
    print(key)

arquitectura-y-urbanismo
diseno-grafico
diseno-industrial
conservatorio-de-musica
cine-y-television
instituto-de-investigaciones-esteticas
artes-plasticas-y-visuales


In [0]:
def url_maker(departamento, docente):
    base_url = "https://losestudiantes.co/universidad-nacional/"
    url ="{}{}/profesores/{}".format(base_url,departamento,docente)
    return(url)



In [0]:
url_administracion_contaduria="http://fce.unal.edu.co/docentesfce/index.php?escuela=1"
html_administracion_contaduria =  requests.get(url_administracion_contaduria).text
arbol = BeautifulSoup(html_administracion_contaduria)
docente_dep = []
for profesor in arbol.find_all("h4",{ "style":"font-size: 16px"}):
    docente_dep.append(normalize(profesor.text))
docentes["administracion-y-contaduria-publica"] = docente_dep

In [0]:
url_economia="http://fce.unal.edu.co/docentesfce/index.php?escuela=2"
html_economia =  requests.get(url_economia).text
arbol = BeautifulSoup(html_economia)
docente_dep = []
for profesor in arbol.find_all("h4",{ "style":"font-size: 16px"}):
  docente_dep.append(normalize(profesor.text))
docentes["economia"] = docente_dep


In [7]:
urls = []
with open("/content/drive/My Drive/url_andes.txt","r") as url_andes:
  arbol = BeautifulSoup(url_andes.read())
  for a in arbol.find_all("a",{"class":"jsx-633353764"}):
    urls.append("https://losestudiantes.co/"+a['href'])
print(len(urls))

3302


In [0]:
import json
json.dump( docentes, open( "profesors.json", 'w' ) )

for k,v in docentes.items():
  for profesor in v:
    urls.append(url_maker(k,profesor))
with open('urls.txt', 'w') as f:
    for item in urls:
        f.write("%s\n" % item)

with open('urls.txt', 'w') as f:
    for item in urls:
        f.write("%s\n" % item)

In [20]:
train_data =[]
if os.path.isfile('reviews.json'):
    # # Read data from file:
      train_data = json.load( open( "reviews.json" ) )
else:
    for url in urls:
        html_review = requests.get(url).text
        arbol = BeautifulSoup(html_review)
        containers = arbol.find_all("li",{"class":"jsx-571610088 post "})  
        for container in containers:
            review = container.find("div",{"class":"jsx-571610088 lineBreak"})
            nota = container.find("span",{"class":"jsx-571610088 numeroStats"})
            if nota is None or review is None:
                continue
            train_data.append((review.text, str(nota.text),url))

'\nfor url in urls:\n  html_review = requests.get(url).text\n  arbol = BeautifulSoup(html_review)\n  containers = arbol.find_all("li",{"class":"jsx-571610088 post "})  \n  for container in containers:\n    # nota = container.find("span",{"class":"jsx-571610088 numeroStats"}).text\n    # if nota <= 0.5:\n    #   nota = 0.5\n    # elif 0.5<nota<=1:\n    #   nota = 1\n    # elif 1<nota<=1.5:\n    #   nota = 1.5\n    # elif 1.5<nota<=2:\n    #   nota = 2\n    # elif 2<nota<=2.5:\n    #   nota = 2.5\n    # elif 2.5<nota<=3:\n    #   nota = 3\n    # elif 3<nota<=3.5:\n    #   nota = 3.5\n    # elif 3.5<nota<=4:\n    #   nota = 4\n    # elif 4<nota<=4.5:\n    #   nota = 4.5\n    # elif 4.5<nota<=5:\n    #   nota = 5\n    review = container.find("div",{"class":"jsx-571610088 lineBreak"})\n    nota = container.find("span",{"class":"jsx-571610088 numeroStats"})\n    if nota is None or review is None:\n      continue\n    train_data.append((review.text,\n                str(nota.text),url))\n'

In [0]:
with open('reviews.txt', 'w') as f:
    for item in train_data:
        f.write("%s --%s\n" % (item[0],item[1]))
json.dump( train_data, open( "reviews.json", 'w', encoding='utf-8' ) )

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd

In [23]:
df = pd.DataFrame(train_data)
print(df[df[1]=="5"].count())
print(df)

0    7677
1    7677
2    7677
dtype: int64
                                                       0  ...                                                  2
0      Es lo peor en serio no explica nada, no es cla...  ...  https://losestudiantes.co/universidad-de-los-a...
1      Todo un amor de profesor, no sé como sea para ...  ...  https://losestudiantes.co/universidad-de-los-a...
2      El CBU es un poco aburrido si no te interesa e...  ...  https://losestudiantes.co/universidad-de-los-a...
3      Es la clase más aburrida que he visto en la un...  ...  https://losestudiantes.co/universidad-de-los-a...
4      Hernando es muy buena persona, en realidad es ...  ...  https://losestudiantes.co/universidad-de-los-a...
...                                                  ...  ...                                                ...
24394  Excelente cátedra. Sus clases están dotadas de...  ...  https://losestudiantes.co/universidad-nacional...
24395  Muy buen profesor, el sesgo es evidente pero a

In [13]:

cv = CountVectorizer(binary=True)
cv.fit(df[0])
X = cv.transform(df[0])

x_train,x_test,y_train , y_test = train_test_split(X,df[1], train_size =  0.75)
print(y_train)

3562       5
18758      5
8903       5
3569       5
1275     2.9
        ... 
4126       5
3621       5
9909     4.5
7589       5
17607    4.3
Name: 1, Length: 18299, dtype: object


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 1]:
  print
  lr = LogisticRegression(solver='newton-cg',C=c,max_iter=1000)
  lr.fit(x_train, y_train)
  accuracy = accuracy_score(y_test, lr.predict(x_test))
  if accuracy >= 0.4:
    print ("Accuracy for C=%s: %s" 
          % (c, accuracy))
logit = LogisticRegression(C=1, solver='lbfgs', n_jobs=-1, random_state=7, max_iter=1000)
log_reg = logit.fit(x_train,y_train)


Accuracy for C=0.05: 0.41114754098360656
Accuracy for C=0.25: 0.4547540983606557
Accuracy for C=0.5: 0.47672131147540986
Accuracy for C=1: 0.4914754098360656


In [15]:
'''
from keras.models import Sequential
from keras.layers import Dense
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=1902, activation='relu'))
model.add(Dense(1902, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, df[1], epochs=150, batch_size=10)
'''

"\nfrom keras.models import Sequential\nfrom keras.layers import Dense\n# define the keras model\nmodel = Sequential()\nmodel.add(Dense(12, input_dim=1902, activation='relu'))\nmodel.add(Dense(1902, activation='relu'))\nmodel.add(Dense(1, activation='sigmoid'))\n# compile the keras model\nmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n# fit the keras model on the dataset\nmodel.fit(X, df[1], epochs=150, batch_size=10)\n"

In [17]:
review="es pesimo profesor, no merece estar en esa universidad"
test =[(review)]
val = cv.transform(test)
print(log_reg.predict(val))

['1']
